# Import Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Basic libraries
#
import random
import time
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Sklearn library
#
from sklearn.preprocessing   import StandardScaler


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Tensorflow library
#
import tensorflow                as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks  import TerminateOnNaN, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils      import plot_model


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# User libraries
#
from utils.metrics            import PEHE, ATE
from utils.Loss               import *
from utils.Modified_DragonNet import *
from utils.kNearestNeighbors  import *
from utils.utils              import MSE
from utils.dataloader         import *

print('[INFO] All libraries were imported')

# Parameters

In [ ]:
Problem = 'Synthetic'
problem_type = 'linear'  # 'linear', 'sin'
train_size = 0.8
n_f, n_i = 10, 5
p = 0.3
seed = 42

metric = 'euclidean'  # 'euclidean', 'chebyshev', 'cityblock'
k = 11
Model = 'Modified-Dragonnet'

MODEL_PATH = f'./checkpoints/{Problem}_{Model}-{metric}-k={k}'



random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
# tf.random.set_seed(seed) # for Tensoflow >= 2.0
tf.random.set_random_seed(seed)

In [ ]:
targeted_regularization    = True # {True, False}

output_dir                 = ''
knob_loss                  = dragonnet_loss_binarycross
ratio                      = 1.
validation_split           = 0.2
batch_size                 = 64
verbose                    = False

metrics = [regression_loss, binary_classification_loss, treatment_accuracy, track_epsilon]

if targeted_regularization:
    loss = make_tarreg_loss(ratio=ratio, dragonnet_loss=knob_loss)
else:
    loss = knob_loss

# Get files

In [ ]:
DataLoader = Synthetic(type=problem_type, size=1000, n_f=n_f, n_i=n_i, p=p, seed=seed)
DataLoader.create_dataset(train_size=0.8)

# Simulations

In [ ]:
# Start timer
#
start1 = time.time()



# Load training data
#
trainX, trainT, trainY, train_potential_Y = DataLoader.get_training_data()

# Load testing data
#
testX, testT, testY, test_potential_Y     = DataLoader.get_testing_data()
#
print('[INFO] Datasets imported')



# Setup scaler for target variable
#
scalerY       = StandardScaler()
trainY_scaled = scalerY.fit_transform( trainY.reshape(-1,1) ).squeeze(-1)



# Setup scaler for inputs
scalerX = StandardScaler()
#
trainX  = scalerX.fit_transform( trainX )
testX   = scalerX.transform( testX )



# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# ***                                     New framework                                     ***
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#
# Include information about the outcomes
knn = KNearestNeighbors(k        = [k], 
                        metrics  = [metric], 
                        verbose  = True)
knn(trainX, trainT, trainY)

knn.calculateDistances( trainX )
train_knn_Y = knn.getEstimatedOutcomes()        

knn.calculateDistances( testX  )
test_knn_Y  = knn.getEstimatedOutcomes()        
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=







# Setup DragonNet
#
dragonnet = make_modified_dragonnet(trainX.shape[1], 0.01)


# Create outputs for DragonNet (concatenate Y & T)
#
yt_train = np.concatenate([trainY_scaled.reshape(-1,1), trainT.reshape(-1,1)], axis = 1)


#
#
# *** Training - Phase I ***
#
#

# Compile network
#
dragonnet.compile(optimizer = Adam(lr=1e-3), 
                    loss      = loss, 
                    metrics   = metrics)

# Setup callbacks
callbacks = [TerminateOnNaN(),
                EarlyStopping(monitor   = 'val_loss', 
                            patience  = 2, 
                            min_delta = 0.),
                ReduceLROnPlateau(monitor   = 'loss', 
                                factor    = 0.5, 
                                patience  = 5, 
                                verbose   = verbose, 
                                mode      = 'auto', 
                                min_delta = 1e-8, 
                                cooldown  = 0, 
                                min_lr    = 0)]

start_time = time.time()

# Training
#
dragonnet.fit([trainX, train_knn_Y[:,0], train_knn_Y[:,1]], yt_train, 
                callbacks        = callbacks,
                validation_split = validation_split,
                epochs           = 100,
                batch_size       = batch_size, 
                verbose          = verbose)


# Save trained network
#
dragonnet.save_weights( MODEL_PATH )

print("[INFO] Training - Phase I - Time %.2f secs" % (time.time() - start_time) )







#
#
# *** Training - Phase II ***
#
#    

# Setup callbacks
#
callbacks = [TerminateOnNaN(),
                EarlyStopping(monitor   = 'val_loss', 
                            patience  = 40, 
                            min_delta = 0.),
                ReduceLROnPlateau(monitor   = 'loss', 
                                factor    = 0.5, 
                                patience  = 5, 
                                verbose   = verbose, 
                                mode      = 'auto',
                                min_delta = 0., 
                                cooldown  = 0, 
                                min_lr    = 0)
]

# Compile network
#
dragonnet.compile(optimizer = SGD(lr=1e-5, momentum=0.9, nesterov=True), 
                    loss      = loss,
                    metrics   = metrics)




start_time = time.time()

# Training
#
dragonnet.fit([trainX, train_knn_Y[:,0], train_knn_Y[:,1]], yt_train, 
                callbacks        = callbacks,
                validation_split = validation_split,
                epochs           = 300,
                batch_size       = batch_size, 
                verbose          = verbose)

print("[INFO] Training - Phase II - Time %.2f secs" % (time.time() - start_time) )





#
#
# *** Predictions ***
#
#       
yt_hat_test  = dragonnet.predict( [testX, test_knn_Y[:,0], test_knn_Y[:,1]] )

# In case Dragonnet's training has not been converged, the trained network from Phase I is reloaded
if ( np.isnan( yt_hat_test ).any() ):
    dragonnet.load_weights( MODEL_PATH )
    yt_hat_test  = dragonnet.predict( [testX, test_knn_Y[:,0], test_knn_Y[:,1]] )

    


# Get predictions
#
test_y_hat = yt_hat_test[:,:2]

# Apply inverse transformation
#
test_y_hat[:,0] = scalerY.inverse_transform( test_y_hat[:,0].reshape(-1,1) )[0]
test_y_hat[:,1] = scalerY.inverse_transform( test_y_hat[:,1].reshape(-1,1) )[0]

# Get propensity score
#
propensity_score = yt_hat_test[:,2]





# Results
Results = dict()
Results['Error_PEHE'] = PEHE(test_potential_Y, test_y_hat)
Results['Error_ATE'] = ATE(test_potential_Y, test_y_hat)  
Results['MSE_0'] = MSE(test_potential_Y[:,0], test_y_hat[:,0])
Results['MSE_1'] = MSE(test_potential_Y[:,1], test_y_hat[:,1])

    
print('[INFO] Error of PEHE and ATE computed')
print('[INFO] Time %.2f\n\n' % (time.time() - start1))

# Feature Importance

## Permutation Feature Importance

In [ ]:
# Set feature names
Features = [f'X{i+1}' for i in range(testX.shape[1])] + ['y0_bar', 'y1_bar']

# Number of Iteration for Permutation-Feature-Importance
nSimulations = 100

In [ ]:
from tqdm import tqdm

Performance = {'Error_PEHE': [], 'Error_ATE': [], 'MSE_0': [], 'MSE_1':  [], 'Feature': [], 'Iteration': []}

# Features
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
for i in range(testX.shape[1]):
    print('Feature: ', Features[i])

    for Sim in tqdm(range(nSimulations)):
        testX_new = testX.copy()
        
        # Shuffle Feature-i
        np.random.shuffle(testX_new[:,0])

        # Get predictions
        yt_hat_test  = dragonnet.predict( [testX_new, test_knn_Y[:,0], test_knn_Y[:,1]] )
        test_y_hat = yt_hat_test[:,:2]

        # Apply inverse transformation
        test_y_hat[:,0] = scalerY.inverse_transform( test_y_hat[:,0].reshape(-1,1) )[0]
        test_y_hat[:,1] = scalerY.inverse_transform( test_y_hat[:,1].reshape(-1,1) )[0]

        # Include Feature performance
        Performance['Error_PEHE'] += [PEHE(test_potential_Y, test_y_hat)]
        Performance['Error_ATE'] += [ATE(test_potential_Y, test_y_hat)  ]
        Performance['MSE_0'] += [MSE(test_potential_Y[:,0], test_y_hat[:,0])]
        Performance['MSE_1'] += [MSE(test_potential_Y[:,1], test_y_hat[:,1])]
        Performance['Feature'] += [Features[i]]
        Performance['Iteration'] += [Sim]




# y0_bar
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
print('Feature: y0_bar')

for Sim in tqdm(range(nSimulations)):
    test_knn_Y_new = test_knn_Y.copy()
    
    # Shuffle Feature-i
    np.random.shuffle(test_knn_Y_new[:,0])

    # Get predictions
    yt_hat_test  = dragonnet.predict( [testX, test_knn_Y_new[:,0], test_knn_Y_new[:,1]] )
    test_y_hat = yt_hat_test[:,:2]

    # Apply inverse transformation
    test_y_hat[:,0] = scalerY.inverse_transform( test_y_hat[:,0].reshape(-1,1) )[0]
    test_y_hat[:,1] = scalerY.inverse_transform( test_y_hat[:,1].reshape(-1,1) )[0]


    # Include Feature performance   
    Performance['Error_PEHE'] += [PEHE(test_potential_Y, test_y_hat)]
    Performance['Error_ATE'] += [ATE(test_potential_Y, test_y_hat)  ]
    Performance['MSE_0'] += [MSE(test_potential_Y[:,0], test_y_hat[:,0])]
    Performance['MSE_1'] += [MSE(test_potential_Y[:,1], test_y_hat[:,1])]
    Performance['Feature'] += ['y0_bar']
    Performance['Iteration'] += [Sim]




# y1_bar
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
print('Feature: y1_bar')

for Sim in tqdm(range(nSimulations)):
    test_knn_Y_new = test_knn_Y.copy()
    
    # Shuffle Feature-i
    np.random.shuffle(test_knn_Y_new[:,1])
    
    # Get predictions
    yt_hat_test  = dragonnet.predict( [testX, test_knn_Y_new[:,0], test_knn_Y_new[:,1]] )
    test_y_hat = yt_hat_test[:,:2]

    # Apply inverse transformation
    test_y_hat[:,0] = scalerY.inverse_transform( test_y_hat[:,0].reshape(-1,1) )[0]
    test_y_hat[:,1] = scalerY.inverse_transform( test_y_hat[:,1].reshape(-1,1) )[0]

    # Include Feature performance
    Performance['Error_PEHE'] += [PEHE(test_potential_Y, test_y_hat)]
    Performance['Error_ATE'] += [ATE(test_potential_Y, test_y_hat)  ]
    Performance['MSE_0'] += [MSE(test_potential_Y[:,0], test_y_hat[:,0])]
    Performance['MSE_1'] += [MSE(test_potential_Y[:,1], test_y_hat[:,1])]
    Performance['Feature'] += ['y1_bar']
    Performance['Iteration'] += [Sim]

In [ ]:
df = pd.DataFrame.from_dict(Performance)

for performanceMetric in ['Error_PEHE', 'Error_ATE', 'MSE_0', 'MSE_1']:
    df[performanceMetric] = df[performanceMetric] - Results[performanceMetric]


df.to_csv(f'PFI/Problem={Problem}-{problem_type}-{p}_Model={Model}_k={k}_metric={metric}.csv', index = False)

## Shap

In [ ]:
import shap
explainer = shap.DeepExplainer(dragonnet, [trainX, train_knn_Y[:,0].reshape(-1,1), train_knn_Y[:,1].reshape(-1,1)])

shap_values = explainer.shap_values([testX, test_knn_Y[:,0].reshape(-1,1), test_knn_Y[:,1].reshape(-1,1)])



concatenate_shap_values = []
for i in range(4):
    concatenate_shap_values += [np.concatenate([shap_values[i][0], shap_values[i][1], shap_values[i][2]], axis=1)]

np.savez(f'SHAP/Problem={Problem}-{problem_type}-{p}_Model={Model}_k={k}_metric={metric}.npz', shap_values=np.array(concatenate_shap_values), Features=np.array(Features))
